In [7]:
import tkinter as tk
from tkinter import messagebox
import cv2
import time

class VideoCaptureApp:
    def __init__(self, window):
        self.window = window
        self.window.title("Tkinter OpenCV Video Capture")
        self.window.attributes('-fullscreen', True)
        
        # Set up video source (webcam)
        self.video_source = 0
        self.vid = cv2.VideoCapture(self.video_source)
        
        # Fullscreen canvas for video display
        self.screen_width = self.window.winfo_screenwidth()
        self.screen_height = self.window.winfo_screenheight()
        self.canvas = tk.Canvas(window, width=self.screen_width, height=self.screen_height - 40)
        self.canvas.pack()

        # Button frame at the bottom
        self.button_frame = tk.Frame(window, height=40, bg='black')
        self.button_frame.pack(fill=tk.X, side=tk.BOTTOM)

        # Record button
        self.btn_record = tk.Button(self.button_frame, text="Record", width=20, command=self.record)
        self.btn_record.pack(side=tk.LEFT, padx=20, pady=10)

        # Close button
        self.btn_close = tk.Button(self.button_frame, text="Close", width=20, command=self.close)
        self.btn_close.pack(side=tk.RIGHT, padx=20, pady=10)

        self.recording = False
        self.out = None
        self.start_time = None
        self.video_file = "output.avi"

        self.delay = 15
        self.update()
        self.window.mainloop()

    def record(self):
        if not self.recording:
            self.recording = True
            self.start_time = time.time()
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            self.out = cv2.VideoWriter(self.video_file, fourcc, 20.0, (int(self.vid.get(3)), int(self.vid.get(4))))
            self.btn_record.config(text="Recording...")
            self.record_video()

    def record_video(self):
        if self.recording:
            ret, frame = self.vid.read()
            if ret:
                if time.time() - self.start_time <= 30:
                    self.out.write(frame)
                    self.photo = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    self.photo = cv2.resize(self.photo, (self.screen_width, self.screen_height - 40))
                    self.photo = tk.PhotoImage(master=self.canvas, data=cv2.imencode('.ppm', self.photo)[1].tobytes())
                    self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
                    self.window.after(self.delay, self.record_video)
                else:
                    self.stop_recording()

    def stop_recording(self):
        if self.recording:
            self.recording = False
            if self.out:
                self.out.release()
            self.out = None
            self.vid.release()  # Release the webcam
            self.btn_record.config(text="Record")
            messagebox.showinfo("Info", "Video recording stopped after 30 seconds.")

    def update(self):
        if not self.recording:
            ret, frame = self.vid.read()
            if ret:
                self.photo = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                self.photo = cv2.resize(self.photo, (self.screen_width, self.screen_height - 40))
                self.photo = tk.PhotoImage(master=self.canvas, data=cv2.imencode('.ppm', self.photo)[1].tobytes())
                self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
        self.window.after(self.delay, self.update)

    def close(self):
        self.window.destroy()

    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()
        if self.out:
            self.out.release()

# Create a window and pass it to the Application object
root = tk.Tk()
app = VideoCaptureApp(root)
